In [1]:
import os
import csv
from countries_languages import country_to_language

import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime

from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name



/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/AssemblyHelpers.py:54: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [2]:
def load_csv_dataset(file_path):
    return pd.read_csv(file_path)

#leagues value
leagues_value = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/Most Updated Edited Transfermarkt Dataset.csv')
#leagues value large 
leagues_value_large = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/most_updated_transfermarkt_dataset.csv')

countries_codes = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/countries_and_codes.csv')

result_names_null_values = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/nullnames_values.csv')
result_names_null_values = list(result_names_null_values['Name'])




/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_56713/3839833153.py:2: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path)


In [3]:
#combine datasets into one 

import pandas as pd
import glob

# Define the path to your directory containing CSV files
directory_path = '/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/lookup_dataset_1/'

# Get a list of all CSV files in the directory
csv_files = glob.glob(directory_path + '*.csv')

# Initialize an empty list to store DataFrames
dfs = []

# Loop through each CSV file
for csv_file in csv_files:
    # Load the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    # Append the DataFrame to the list
    dfs.append(df)

# Concatenate all DataFrames into one
combined_df = pd.concat(dfs, ignore_index=True)

# Now you have a single DataFrame containing data from all CSV files


In [3]:
#combined_df.to_csv('large_dataset_1.csv')

In [4]:
combined_df_failed = combined_df[combined_df['Status'] == 'FAIL']
combined_df_failed['Lookup Return Case'].value_counts().head(25)

Lookup Return Case
Null Name                                                                                                                                                          8164
Lookup Required                                                                                                                                                    3534
not done yet                                                                                                                                                       2844
Online Lookup Required                                                                                                                                             1814
Was in  DB before or after season                                                                                                                                  1255
asswipe was in DB in before or after season                                                                                                  

# LOOKUP REQUIRED

In [10]:
combined_df_failed_lookup_required = combined_df_failed[combined_df_failed['Lookup Return Case'] == 'Lookup Required']

In [11]:
#lookup required - helper functions
from fuzzywuzzy import process

def find_close_matches(this_jersey, dataset_nationality, threshold=90):
    """
    Find close matches of `this_jersey` in `dataset_nationality` using Levenshtein distance.

    Args:
    - this_jersey (str): The string to find close matches for.
    - dataset_nationality (list): List of strings to search for close matches in.
    - threshold (int): Minimum similarity score required for a match (default is 90).

    Returns:
    - List of strings from `dataset_nationality` that are close matches to `this_jersey`.
    """
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]


def find_close_matches_variable(this_jersey, dataset_nationality, threshold):
    """
    Find close matches of `this_jersey` in `dataset_nationality` using Levenshtein distance.

    Args:
    - this_jersey (str): The string to find close matches for.
    - dataset_nationality (list): List of strings to search for close matches in.
    - threshold (int): Minimum similarity score required for a match (default is 90).

    Returns:
    - List of strings from `dataset_nationality` that are close matches to `this_jersey`.
    """
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]

# Example usage:
# this_jersey = "United States"
# dataset_nationality = ["United States", "United Kingdom", "Canada", "Germany", "France"]
# close_matches = find_close_matches(this_jersey, dataset_nationality)
# print(close_matches)

def rearrange_name_asian(this_jersey):
    """
    Rearrange the name in `this_jersey` so that the last token comes first followed by a space.

    Args:
    - this_jersey (str): The name to rearrange.

    Returns:
    - Rearranged name string.
    """
    tokens = this_jersey.split()
    rearranged_name = tokens[-1] + " " + " ".join(tokens[:-1])
    return rearranged_name

import unicodedata


def normalize_name(name):
    """
    Normalize the given name by removing dashes, hyphens, apostrophes, and backticks,
    and replacing characters with accents with their unaccented counterparts.

    Args:
    - name (str): The name to normalize.

    Returns:
    - Normalized name string.
    """
    # Remove dashes, hyphens, apostrophes, and backticks
    name = re.sub(r'[-\'`]', '', name)

    # Replace characters with accents with their unaccented counterparts
    name = ''.join(char for char in unicodedata.normalize('NFD', name) if unicodedata.category(char) != 'Mn')

    return name

def switch_yi(string):
    if string.endswith('yi'):
        switched_string = string[:-2] + 'iy'
        return switched_string
    elif string.endswith('iy'):
        switched_string = string[:-2] + 'yi'
        return switched_string
    else:
        return string

In [12]:
for i in range(0, len(leagues_value)):
    if leagues_value.at[i, 'Nationality'] == 'China':
        leagues_value.at[i, 'Team 1 Code'] = 'CN'
    elif leagues_value.at[i, 'Nationality'] == 'Democratic Republic of the Congo':
        leagues_value.at[i, 'Team 1 Code'] = 'DR'


In [13]:
combined_df_failed_lookup_required['Status II'] = ''


/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_56713/3149873880.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df_failed_lookup_required['Status II'] = ''


In [14]:
dataframe_matches = pd.DataFrame()

dataframe_MULTIS = pd.DataFrame()

for index, row in combined_df_failed_lookup_required.iterrows(): 
    name_list_matches = []

    this_jersey = row['ORIGINAL JERSEY']
    
    nationality = row['Nationality']
    nationality_code = row['Team Country Code']
    if nationality_code == 'BVI':
        nationality_code = ' BVI'
    country_name = countries_codes[countries_codes[' Code'] == nationality_code]['Country'].unique()[0]
    if nationality_code == ' DR':
        nationality_code = 'DR'
    season = row['Season']
    match_date = row['Date']
    dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == nationality_code]['Name'].unique()

    if this_jersey.endswith('yi'):
        this_jersey = switch_yi(this_jersey)
    elif this_jersey.endswith('iy'):
        this_jersey = switch_yi(this_jersey)
    
    if nationality == "Vietnam":
        #if they are from vietnam try to 
        vietnames_matching = []
        for name in dataset_nationality:
            if unidecode(name) == this_jersey:
                vietnames_matching.append(name)
        if len(vietnames_matching) == 1:
            print('✓ - Viet guy WORKING', rearrange_name_asian(this_jersey), name)
            new_row = {
                'Jersey Name': this_jersey,
                'Name Match': vietnames_matching,
                'Date': match_date,
                'Nationality': nationality,
                'Country Code': nationality_code,
                'Match ID': row['Match ID'], 
                'Competition': row['Competition'], 
                'Match': row['Match'],
            }
            dataframe_matches = pd.concat([dataframe_matches, pd.DataFrame([new_row])], ignore_index=True)
            
        #print('Viet guy')
        0==0
    elif (nationality_code == 'KR') | (nationality_code == 'CN') | (nationality_code == 'TW') | (nationality_code == 'SG') | (nationality_code == 'KP'):
        #try to flip the names and find it with that 
        close_match_names = find_close_matches(rearrange_name_asian(this_jersey), dataset_nationality)
        if len(close_match_names) == 1:
            combined_df_failed_lookup_required.at[index, 'Status II'] = 'Working'
            print('✓ - Backwards name guy WORKING', rearrange_name_asian(this_jersey), close_match_names)
            new_row = {
                'Jersey Name': this_jersey,
                'Name Match': close_match_names,
                'Date': match_date,
                'Nationality': nationality,
                'Country Code': nationality_code,
                'Match ID': row['Match ID'], 
                'Competition': row['Competition'], 
                'Match': row['Match'],
            }
            dataframe_matches = pd.concat([dataframe_matches, pd.DataFrame([new_row])], ignore_index=True)
        elif len(close_match_names) >= 2:
            realmatches = []
            for close_match in close_match_names:
                if this_jersey == close_match:
                    realmatches.append(close_match)
            if len(realmatches) == 1: 
                combined_df_failed_lookup_required.at[index, 'Status II'] = 'Working'
                print('✓ - Backwards name guy WORKING MULTI FILTER', rearrange_name_asian(this_jersey), realmatches)
                new_row = {
                    'Jersey Name': this_jersey,
                    'Name Match': realmatches,
                    'Date': match_date,
                    'Nationality': nationality,
                    'Country Code': nationality_code,
                    'Match ID': row['Match ID'], 
                    'Competition': row['Competition'], 
                    'Match': row['Match'],
                }
                dataframe_matches = pd.concat([dataframe_matches, pd.DataFrame([new_row])], ignore_index=True)
            else:
                print('x - Backwards name guy MULTI', rearrange_name_asian(this_jersey), close_match_names)
        else:
            print('x - Backwards name guy 0', rearrange_name_asian(this_jersey), nationality)
    else:
        this_jersey_tokens = this_jersey.split(' ')
        if len(this_jersey_tokens) >= 2:
            if len(this_jersey_tokens[0]) <= 2: 
                    #if token 0 length is one or two and there are multiple tokens – that's the F Last
                    #starts with an initial end with a name 
                    last_F_matches = []
                    for name in dataset_nationality:
                        if (name.startswith(this_jersey_tokens[0])) & (name.endswith(this_jersey_tokens[-1])):
                            last_F_matches.append(name)
                        elif (name.startswith(this_jersey_tokens[0])) & (name.endswith(unidecode(this_jersey_tokens[-1]))):
                            last_F_matches.append(name)
                    if len(last_F_matches) == 1:
                        combined_df_failed_lookup_required.at[index, 'Status II'] = 'Working'
                        print('✓ - F Last Case WORKING', this_jersey, last_F_matches)
                        new_row = {
                            'Jersey Name': this_jersey,
                            'Name Match': last_F_matches,
                            'Date': match_date,
                            'Nationality': nationality,
                            'Country Code': nationality_code,
                            'Match ID': row['Match ID'], 
                            'Competition': row['Competition'], 
                            'Match': row['Match'],
                        }
                        dataframe_matches = pd.concat([dataframe_matches, pd.DataFrame([new_row])], ignore_index=True)
                    elif len(last_F_matches) >= 2:
                        print('x - F Last Case MULTIPLE STILL', this_jersey, last_F_matches)
                        ###MULTI HANDLE 
                        new_row_MULTI = {
                            'Jersey Name': this_jersey,
                            'Name Match': last_F_matches,
                            'Date': match_date,
                            'Nationality': nationality,
                            'Country Code': nationality_code,
                            'Match ID': row['Match ID'], 
                            'Competition': row['Competition'], 
                            'Match': row['Match'],
                        }
                        dataframe_MULTIS = pd.concat([dataframe_MULTIS, pd.DataFrame([new_row_MULTI])], ignore_index=True)
                    else:
                        print('x - F Last Case 0 LEFT', this_jersey, last_F_matches)
            elif len(this_jersey_tokens[-1]) <= 2: 
                    #if token 0 length is one or two and there are multiple tokens – that's the Last F
                    #starts with a name end with a initial  
                    last_F_matches = []
                    for name in dataset_nationality:
                        if (name.startswith(this_jersey_tokens[-1])) & (name.endswith(this_jersey_tokens[0])):
                            last_F_matches.append(name)
                    if len(last_F_matches) == 1:
                        combined_df_failed_lookup_required.at[index, 'Status II'] = 'Working'
                        print('✓ - Last F Case WORKING', this_jersey, last_F_matches)
                        new_row = {
                            'Jersey Name': this_jersey,
                            'Name Match': last_F_matches,
                            'Date': match_date,
                            'Nationality': nationality,
                            'Country Code': nationality_code,
                            'Match ID': row['Match ID'], 
                            'Competition': row['Competition'], 
                            'Match': row['Match'],
                        }
                        dataframe_matches = pd.concat([dataframe_matches, pd.DataFrame([new_row])], ignore_index=True)
                    elif len(last_F_matches) >= 2:
                        print('x - Last F Case MULTIPLE STILL', this_jersey, last_F_matches)
                        ###MULTI HANDLE - DONE
                        new_row_MULTI = {
                            'Jersey Name': this_jersey,
                            'Name Match': last_F_matches,
                            'Date': match_date,
                            'Nationality': nationality,
                            'Country Code': nationality_code,
                            'Match ID': row['Match ID'], 
                            'Competition': row['Competition'], 
                            'Match': row['Match'],
                        }
                        dataframe_MULTIS = pd.concat([dataframe_MULTIS, pd.DataFrame([new_row_MULTI])], ignore_index=True)
                    else:
                        print('x - Last F Case 0 LEFT', this_jersey, last_F_matches)
            else:
                name_list_matches = []
                0==0 #check if this_jersey starts with any of the names in dataset_nationality 
                for name in dataset_nationality:
                    if this_jersey.startswith(name):
                        name_list_matches.append(name)
                        #print('found a match here', this_jersey, name, match_date)
                    elif unidecode(this_jersey).startswith(name):
                        name_list_matches.append(name)
                        #print('found a unidecode match here', this_jersey, name, match_date)
                    elif this_jersey.startswith(unidecode(name)):
                        name_list_matches.append(name)
                        #print('found a unidecode match here ii', this_jersey, name, match_date)
                if len(name_list_matches) == 1:
                    #RETURN HERE
                    combined_df_failed_lookup_required.at[index, 'Status II'] = 'Working'
                    print('✓ - start with match case', this_jersey, name_list_matches, match_date)
                    new_row = {
                        'Jersey Name': this_jersey,
                        'Name Match': name_list_matches,
                        'Date': match_date,
                        'Nationality': nationality,
                        'Country Code': nationality_code,
                        'Match ID': row['Match ID'], 
                        'Competition': row['Competition'], 
                        'Match': row['Match'],
                    }
                    dataframe_matches = pd.concat([dataframe_matches, pd.DataFrame([new_row])], ignore_index=True)
                elif len(name_list_matches) >= 2:
                    print('x - multiple names found', this_jersey, name_list_matches)
                    ###MULTI HANDLE 
                    new_row_MULTI = {
                        'Jersey Name': this_jersey,
                        'Name Match': name_list_matches,
                        'Date': match_date,
                        'Nationality': nationality,
                        'Country Code': nationality_code,
                        'Match ID': row['Match ID'], 
                        'Competition': row['Competition'], 
                        'Match': row['Match'],
                    }
                    dataframe_MULTIS = pd.concat([dataframe_MULTIS, pd.DataFrame([new_row_MULTI])], ignore_index=True)
                else:
                    close_match_names = find_close_matches(this_jersey, dataset_nationality)
                    if len(close_match_names) == 1:
                        combined_df_failed_lookup_required.at[index, 'Status II'] = 'Working'
                        print('✓ - close match found', close_match_names, this_jersey, nationality, match_date, nationality_code)
                        new_row = {
                            'Jersey Name': this_jersey,
                            'Name Match': close_match_names,
                            'Date': match_date,
                            'Nationality': nationality,
                            'Country Code': nationality_code,
                            'Match ID': row['Match ID'], 
                            'Competition': row['Competition'], 
                            'Match': row['Match'],
                        }
                        dataframe_matches = pd.concat([dataframe_matches, pd.DataFrame([new_row])], ignore_index=True)
                    elif len(close_match_names) >= 2:
                        print('x - multiple matches found', close_match_names, this_jersey, nationality, match_date, nationality_code)
                        ###MULTI HANDLE 
                        new_row_MULTI = {
                            'Jersey Name': this_jersey,
                            'Name Match': close_match_names,
                            'Date': match_date,
                            'Nationality': nationality,
                            'Country Code': nationality_code,
                            'Match ID': row['Match ID'], 
                            'Competition': row['Competition'], 
                            'Match': row['Match'],
                        }
                        dataframe_MULTIS = pd.concat([dataframe_MULTIS, pd.DataFrame([new_row_MULTI])], ignore_index=True)
                    else:
                        #print('no matches found', this_jersey, nationality, match_date, nationality_code)

                        this_jersey_dashes_accents_removed = normalize_name(this_jersey)
                        close_match_names_accents_removed = find_close_matches(this_jersey_dashes_accents_removed, dataset_nationality)
                        if len(close_match_names) == 1:
                            combined_df_failed_lookup_required.at[index, 'Status II'] = 'Working'
                            print('✓ - accents removed. close match found', close_match_names, this_jersey, nationality, match_date, nationality_code)
                            new_row = {
                                'Jersey Name': this_jersey,
                                'Name Match': close_match_names,
                                'Date': match_date,
                                'Nationality': nationality,
                                'Country Code': nationality_code,
                                'Match ID': row['Match ID'], 
                                'Competition': row['Competition'], 
                                'Match': row['Match'],
                            }
                            dataframe_matches = pd.concat([dataframe_matches, pd.DataFrame([new_row])], ignore_index=True)
                        elif len(close_match_names) >= 2:
                            ###MULTI HANDLE 
                            print('x - accents removed. multiple matches found', close_match_names, this_jersey, nationality, match_date, nationality_code)
                            new_row_MULTI = {
                                'Jersey Name': this_jersey,
                                'Name Match': close_match_names,
                                'Date': match_date,
                                'Nationality': nationality,
                                'Country Code': nationality_code,
                                'Match ID': row['Match ID'], 
                                'Competition': row['Competition'], 
                                'Match': row['Match'],
                            }
                            dataframe_MULTIS = pd.concat([dataframe_MULTIS, pd.DataFrame([new_row_MULTI])], ignore_index=True)
                        else:
                            print('x - accents removed. no matches found', this_jersey, nationality, match_date, nationality_code)
        else:
            for name in dataset_nationality:
                if name.endswith(this_jersey):
                    name_list_matches.append(name) 
                elif unidecode(name).endswith(this_jersey):
                    name_list_matches.append(name)
                    
                elif name.endswith(unidecode(this_jersey)):
                    name_list_matches.append(name)
                    
            if len(name_list_matches) == 1:
                #RETURN HERE
                combined_df_failed_lookup_required.at[index, 'Status II'] = 'Working'
                print('✓ - SINGLE NAME match case', this_jersey, name_list_matches, match_date)
                new_row = {
                    'Jersey Name': this_jersey,
                    'Name Match': name_list_matches,
                    'Date': match_date,
                    'Nationality': nationality,
                    'Country Code': nationality_code,
                    'Match ID': row['Match ID'], 
                    'Competition': row['Competition'], 
                    'Match': row['Match'],
                }
                dataframe_matches = pd.concat([dataframe_matches, pd.DataFrame([new_row])], ignore_index=True)
            else:
                0==0

✓ - F Last Case WORKING D Tošić ['Dusko Tosic']
✓ - F Last Case WORKING J Næs ['Jónas Tór Naes']
x - F Last Case 0 LEFT B Jógvansson-Hansen []
✓ - F Last Case WORKING S Kjær ['Simon Kjaer']
✓ - F Last Case WORKING E Višća ['Edin Visca']
✓ - F Last Case WORKING A Gaţcan ['Alexandru Gatcan']
✓ - F Last Case WORKING A Gaţcan ['Alexandru Gatcan']
✓ - F Last Case WORKING S Kjær ['Simon Kjaer']
✓ - F Last Case WORKING E Bičakčić ['Ermin Bicakcic']
✓ - F Last Case WORKING D Subašić ['Danijel Subasic']
✓ - F Last Case WORKING S Kjær ['Simon Kjaer']
✓ - F Last Case WORKING J Næs ['Jónas Tór Naes']
✓ - Backwards name guy WORKING Junling Yan ['Junling Yan']
✓ - Backwards name guy WORKING Zhuniy Gao ['Zhunyi Gao']
✓ - Backwards name guy WORKING Linpeng Zhang ['Linpeng Zhang']
✓ - Backwards name guy WORKING Chenjie Zhu ['Chenjie Zhu']
✓ - Backwards name guy WORKING MULTI FILTER Zheng Zheng ['Zheng Zheng']
✓ - Backwards name guy WORKING Yang Liu ['Yang Liu']
✓ - Backwards name guy WORKING Hao Junmin

In [15]:
dataframe_matches

,Jersey Name,Name Match,Date,Nationality,Country Code,Match ID,Competition,Match
0,D Tošić,[Dusko Tosic],09-10-2017,Serbia,RS,3136,"World Cup Qualifiers, Europe, 2018",Serbia vs Georgia
1,J Næs,[Jónas Tór Naes],13-11-2016,Faroe Islands,FO,3137,"World Cup Qualifiers, Europe, 2018",Switzerland vs Faroe Islands
2,S Kjær,[Simon Kjaer],01-09-2017,Denmark,DK,3138,"World Cup Qualifiers, Europe, 2018",Denmark vs Poland
3,E Višća,[Edin Visca],06-09-2016,Bosnia-Herzegovina-Herzegovina,BA,3139,"World Cup Qualifiers, Europe, 2018",Bosnia-Herzegovina-Herzegovina vs Estonia
4,A Gaţcan,[Alexandru Gatcan],12-11-2016,Moldova,MD,3140,"World Cup Qualifiers, Europe, 2018",Georgia vs Moldova
...,...,...,...,...,...,...,...,...
1178,I ehić,[Ibrahim Sehic],13-11-2021,Bosnia-Herzegovina-Herzegovina,BA,3269,"World Cup Qualifiers, Europe, 2022",Bosnia-Herzegovina-Herzegovina vs Finland
1179,Maliy,[Sergey Maliy],06.09.18,Kazakhstan,KZ,1488,Nations League 2018-2020,Kazakhstan vs Georgia
1180,Maliy,[Sergey Maliy],13.10.18,Kazakhstan,KZ,1493,Nations League 2018-2020,Latvia vs Kazakhstan
1181,Maliy,[Sergey Maliy],15.11.18,Kazakhstan,KZ,1496,Nations League 2018-2020,Kazakhstan vs Latvia


In [16]:
combined_df_failed_lookup_required[combined_df_failed_lookup_required['Status II'] != 'Working']['ORIGINAL JERSEY'].value_counts().head(50)

ORIGINAL JERSEY
Karavayev              21
Natkho                 21
Drahun                 19
Nguyn Tin Linh         17
D Dakonam              16
Nguyn Tun Anh          16
Bùi Tin Dng            15
Boženík                14
A Amanow               14
Damașcan               14
D Hérelle              13
A-nan P                13
N Tolo                 12
Pico                   12
O Færø                 12
C Mafoumbi             12
R Gomez                12
M Woeber               11
Zheng Zhi              11
Ai Kesen               11
Jiang Guangtai         11
R Yusef                11
Khacheridi Yevhen      11
Hahgverdi              11
Bapasy                 11
Gviliya                11
Y Kavalyow             10
F Niţă                 10
M Baudry               10
K Panayi               10
CZeng                   9
S Abaul                 9
Faisal Saleh            9
D NDinga                9
T Alcantara             9
C Andriamahitsinoro     9
K Parsemain             9
A Al Faz              

In [17]:
combined_df_failed_2 = combined_df_failed_lookup_required[combined_df_failed_lookup_required['Status II'] != 'Working']
combined_df_failed_2['Status III'] = ''

/var/folders/vw/z2xv57_n70l_szw_fr80j5km0000gn/T/ipykernel_56713/2932840586.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined_df_failed_2['Status III'] = ''


In [18]:
def threshold_player_match(this_jersey, dataset_nationality):
    THRESHOLD_NUM = 89
    #stops when it returns a name.
    #if it doesnt find a match keep lowering the threshold until you find a match
    #but if you get to threshold of like 50 first you stop and just abandon ship    

    # Loop until someone is found or threshold goes below 50
    while THRESHOLD_NUM >= 50:
        matches = find_close_matches_variable(this_jersey, dataset_nationality, THRESHOLD_NUM)
        if matches:
            print(f"Player is {this_jersey}. Found matches: {matches}. threshold is {THRESHOLD_NUM}")
            return matches, THRESHOLD_NUM
            #break
        else:
            THRESHOLD_NUM -= 1

    # If threshold reaches below 50 without finding any matches
    if THRESHOLD_NUM < 50:
        print("No matches found even with the lowest threshold. jersey was ", this_jersey)
        return 0, 50

def filter_matches_by_initial_and_name_start(this_jersey_tokens, matches):
    """
    Filter matches based on whether at least one token starts with the initial and at least one token starts with 
    the first character of the name, with additional conditions based on the tokens in this_jersey_tokens.

    Args:
    - this_jersey_tokens (list): List containing the initial and name parts of the jersey.
    - matches (list): List of matches to filter.

    Returns:
    - List of filtered matches.
    """
    filtered_matches = []
    
    # Check if this_jersey_tokens has multiple tokens
    if len(this_jersey_tokens) > 1:
        initial_starts_with = this_jersey_tokens[0]
        name_starts_with = this_jersey_tokens[1][0]

        # Additional logic for handling 'H' and 'Kh'
        if name_starts_with == 'H':
            if any(token.startswith('Kh') for token in this_jersey_tokens[1:]):
                name_starts_with = 'Kh'

        # Additional logic for handling multiple tokens
        for token in this_jersey_tokens[1:]:
            if token[0] != name_starts_with:
                initial_starts_with = None
                break
        
        for match in matches:
            match_tokens = match.split()
            initial_found = False
            name_start_found = False

            # Check if at least one token starts with the initial
            if initial_starts_with:
                for token in match_tokens:
                    if token.startswith(initial_starts_with):
                        initial_found = True
                        break

            # Check if at least one token starts with the first character of the name
            for token in match_tokens:
                if token.startswith(name_starts_with) or (name_starts_with == 'H' and token.startswith('Kh')):
                    name_start_found = True
                    break

            # Check if any token starts with any character of the hyphenated name
            if '-' in this_jersey_tokens[1]:
                hyphenated_parts = this_jersey_tokens[1].split('-')
                for part in hyphenated_parts:
                    for token in match_tokens:
                        if token.startswith(part[0]):
                            name_start_found = True
                            break

            # If both conditions are met, add the match to filtered_matches
            if initial_found and name_start_found:
                filtered_matches.append(match)

    else:  # Handle the case where this_jersey_tokens has only one token
        for match in matches:
            match_tokens = match.split()
            initial_found = False
            name_start_found = False

            # Check if at least one token starts with the initial
            for token in match_tokens:
                if token.startswith(this_jersey_tokens[0]):
                    initial_found = True
                    break

            # Check if at least one token starts with the first character of the name
            for token in match_tokens:
                if token.startswith(this_jersey_tokens[0][0]):
                    name_start_found = True
                    break

            # Check if any token starts with any character of the hyphenated name
            if '-' in this_jersey_tokens[0]:
                hyphenated_parts = this_jersey_tokens[0].split('-')
                for part in hyphenated_parts:
                    for token in match_tokens:
                        if token.startswith(part[0]):
                            name_start_found = True
                            break

            # If both conditions are met, add the match to filtered_matches
            if initial_found and name_start_found:
                filtered_matches.append(match)

    return filtered_matches

# this_jersey_tokens = ['R', 'Hojayev']
# matches = ['Resul Khozhaev']
# print(filter_matches_by_initial_and_name_start_new(this_jersey_tokens, matches))

def check_last_token_match(this_jersey, matches):
    """
    Check if the last token of each match matches the jersey with variations like accents or substitutions.

    Args:
    - this_jersey (str): The original name being searched for.
    - matches (list): Set of names matching the jersey.

    Returns:
    - List of matches where the last token matches the jersey with variations.
    """
    def remove_accents(input_str):
        """
        Remove accents from characters in a string.
        """
        nfkd_form = unicodedata.normalize('NFKD', input_str)
        return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    def apply_substitutions(input_str):
        """
        Apply specific character substitutions in a string.
        """
        substitutions = {'g': 'h', 'ij': 'y', 'zh': 'j', 'ae': 'aye', 'H': 'Kh'}
        for key, value in substitutions.items():
            input_str = input_str.replace(key, value)
        return input_str

    jersey_last_token = this_jersey.split()[-1]

    matching_matches = []
    for match in matches:
        match_last_token = match.split()[-1]

        # Check if the last token matches the jersey (ignoring accents)
        if remove_accents(match_last_token) == remove_accents(jersey_last_token):
            matching_matches.append(match)
        else:
            # Check if the last token matches the jersey with specific character substitutions
            if apply_substitutions(match_last_token) == apply_substitutions(jersey_last_token):
                matching_matches.append(match)

    return matching_matches

def filter_multi_word_matches_by_jersey_tokens(jersey_tokens, potential_matches):
    filtered_matches = []

    for match in potential_matches:
        # Split each match into tokens
        match_tokens = match.split()

        # Count the number of tokens that start with each character in jersey_tokens
        match_token_start_chars = {token[0] for token in match_tokens}

        # Check if the match contains at least one token for each character in jersey_tokens
        if all(token[0] in match_token_start_chars for token in jersey_tokens):
            filtered_matches.append(match)

    return filtered_matches




In [19]:
success_tally = 0
multi_tally = 0
fail_tally = 0
weird_indexes = []
dataframe_matches_2 = pd.DataFrame()
for index, row in combined_df_failed_2.iterrows():
    this_row_success_tally = 0
    this_row_multi_tally = 0
    this_row_fail_tally = 0
# for i in range(0, 1):
#     index = 2348
#     row = combined_df_failed_2.iloc[index]

    name_list_matches = []

    this_jersey = row['ORIGINAL JERSEY']
    
    nationality = row['Nationality']
    nationality_code = row['Team Country Code']
    if nationality_code == 'BVI':
        nationality_code = ' BVI'
    country_name = countries_codes[countries_codes[' Code'] == nationality_code]['Country'].unique()[0]
    if nationality_code == ' DR':
        nationality_code = 'DR'
    season = row['Season']
    match_date = row['Date']
    dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == nationality_code]['Name'].unique()

    if this_jersey.endswith('yi'):
        this_jersey = switch_yi(this_jersey)
    elif this_jersey.endswith('iy'):
        this_jersey = switch_yi(this_jersey)

    name_type = ''
    matches, threshold_num = threshold_player_match(this_jersey, dataset_nationality)

    this_jersey_tokens = this_jersey.split(' ')
    if len(this_jersey_tokens) >= 2:
        if len(this_jersey_tokens[0]) == 1: #initial + name
            name_type = 'Initial, Name'
            if matches == 0:
                this_name = ' '.join(this_jersey_tokens[1:]).strip()
                matches, threshold_num = threshold_player_match(this_name, dataset_nationality) #do a closest match search for the NAME without the initial 
            if matches == 0:
                print(this_jersey, 'No Matches Found')
                #FAIL CASE HERE
                fail_tally += 1
                this_row_fail_tally += 1
            elif len(matches) >= 1:#check that at least 1 token in each match starts with the initial and make sure at least 1 token in each match starts with what the NAME started with
                matches_filtered = filter_matches_by_initial_and_name_start(this_jersey_tokens, matches)
                
                if len(matches_filtered) >= 2:
                    match_set = matches_filtered
                    multi_tally += 1
                    this_row_multi_tally += 1
                    #MULTI CASE HERE
                elif len(matches_filtered) == 1:
                    match_single = matches_filtered[0]
                    success_tally += 1
                    this_row_success_tally += 1
                    #SUCCESS CASE HERE
                    combined_df_failed_2.at[index, 'Status III'] = 'Working'
                    new_row = {
                        'Jersey Name': this_jersey,
                        'Name Match': match_single,
                        'Date': match_date,
                        'Nationality': nationality,
                        'Country Code': nationality_code,
                        'Match ID': row['Match ID'], 
                        'Competition': row['Competition'], 
                        'Match': row['Match'],
                    }
                    dataframe_matches_2 = pd.concat([dataframe_matches_2, pd.DataFrame([new_row])], ignore_index=True)
                    
                else:
                    print(f'filtering by last name and name start left nobody. player was {this_jersey} and matches WERE  {matches_filtered}')
                    fail_tally += 1
                    this_row_fail_tally += 1
                    #FAIL CASE HERE
            else:
                0==0
                print(this_jersey, 'wtf happened here')
                fail_tally += 1
                this_row_fail_tally += 1
        else: #no initials – more than 1 word in the name
            name_type = 'Multiple Words'#the jersey is multiple words 
            if matches == 0:
                print(f'OG Jersey is {this_jersey}. there are multiple words. 0 matches case') #probably try to use just one of the words from the original name and see if that returns a single guy
            else: #there are matches
                matches_filtered = filter_multi_word_matches_by_jersey_tokens(this_jersey_tokens, matches)
                if len(matches_filtered) >= 2:
                    match_set = matches_filtered
                    multi_tally += 1
                    this_row_multi_tally += 1
                    #MULTI CASE HERE
                elif len(matches_filtered) == 1:
                    match_single = matches_filtered[0]
                    success_tally += 1
                    this_row_success_tally += 1
                    #SUCCESS CASE HERE
                    combined_df_failed_2.at[index, 'Status III'] = 'Working'
                    new_row = {
                        'Jersey Name': this_jersey,
                        'Name Match': match_single,
                        'Date': match_date,
                        'Nationality': nationality,
                        'Country Code': nationality_code,
                        'Match ID': row['Match ID'], 
                        'Competition': row['Competition'], 
                        'Match': row['Match'],
                    }
                    dataframe_matches_2 = pd.concat([dataframe_matches_2, pd.DataFrame([new_row])], ignore_index=True)
                    
                else:
                    print(f'OG Jersey is {this_jersey}. multi-word match case filtering left 0 people.') #no  matches left.
                    fail_tally += 1
                    this_row_fail_tally += 1
                    #FAIL CASE HERE
    else: #just a single name
        name_type = 'Single Name'
        if matches == 0: #just one name, and that match didn't work with anybody.
            #FAIL CASE HERE
            print('single name, no matches', this_jersey, matches, nationality)
            fail_tally += 1
            this_row_fail_tally += 1
        else:
            #make sure the last name of each match (final token) matches the jersey 
            matches_filtered = check_last_token_match(this_jersey, matches)

            if matches_filtered:
                #SUCCESS CASE HERE
                success_tally += 1
                this_row_success_tally += 1
                combined_df_failed_2.at[index, 'Status III'] = 'Working'
                new_row = {
                    'Jersey Name': this_jersey,
                    'Name Match': matches_filtered,
                    'Date': match_date,
                    'Nationality': nationality,
                    'Country Code': nationality_code,
                    'Match ID': row['Match ID'], 
                    'Competition': row['Competition'], 
                    'Match': row['Match'],
                }
                dataframe_matches_2 = pd.concat([dataframe_matches_2, pd.DataFrame([new_row])], ignore_index=True)
            else:
                fail_tally += 1 
                this_row_fail_tally += 1
       
    if this_row_fail_tally == 0 and this_row_multi_tally == 0 and this_row_success_tally == 0:
        weird_indexes.append(index)



        

    #maybe also pass in the names it found originalyl but that it got filtered out as BAD NAMES
    #like how low does the threshold have to drop to get another name in the matches that is not one of the BAD NAMES

#in any of these cases when filtering it leads to 0 matches. 
#find a way to say "if you have to drop the threshold down by like 20 to find a single other person in the dataset, call that good"




Player is B Jógvansson-Hansen. Found matches: ['Bárdur Hansen', 'Hedin Hansen']. threshold is 86
Player is E Amantegui. Found matches: ['Pawee Tanthatemee']. threshold is 50
filtering by last name and name start left nobody. player was E Amantegui and matches WERE  []
Player is W Sulaeman. Found matches: ['Abdul Rahman']. threshold is 55
filtering by last name and name start left nobody. player was W Sulaeman and matches WERE  []
Player is Ala Addin Mahdi. Found matches: ['Abdulmuain Al-Jarshi']. threshold is 60
OG Jersey is Ala Addin Mahdi. multi-word match case filtering left 0 people.
Player is W Ylýasow. Found matches: ['Yhlas Saparmammedov']. threshold is 61
filtering by last name and name start left nobody. player was W Ylýasow and matches WERE  []
Player is R Hojayev. Found matches: ['Resul Khozhaev']. threshold is 67
Player is A Amanow. Found matches: ['Arslanmurad Amanov']. threshold is 68
Player is Y Lee. Found matches: ['Il-gwang Yun', 'Hyok-chol O', 'Chol-myong Ri', 'Hyok-c

In [20]:
dataframe_matches = pd.concat([dataframe_matches, dataframe_matches_2], ignore_index=True)

In [30]:
dataframe_matches
#dataframe_matches.to_csv('lookup_required_matches.csv')

,Jersey Name,Name Match,Date,Nationality,Country Code,Match ID,Competition,Match
0,D Tošić,[Dusko Tosic],09-10-2017,Serbia,RS,3136,"World Cup Qualifiers, Europe, 2018",Serbia vs Georgia
1,J Næs,[Jónas Tór Naes],13-11-2016,Faroe Islands,FO,3137,"World Cup Qualifiers, Europe, 2018",Switzerland vs Faroe Islands
2,S Kjær,[Simon Kjaer],01-09-2017,Denmark,DK,3138,"World Cup Qualifiers, Europe, 2018",Denmark vs Poland
3,E Višća,[Edin Visca],06-09-2016,Bosnia-Herzegovina-Herzegovina,BA,3139,"World Cup Qualifiers, Europe, 2018",Bosnia-Herzegovina-Herzegovina vs Estonia
4,A Gaţcan,[Alexandru Gatcan],12-11-2016,Moldova,MD,3140,"World Cup Qualifiers, Europe, 2018",Georgia vs Moldova
...,...,...,...,...,...,...,...,...
1503,Damașcan,[Vitalie Damaşcan],12.10.18,Moldova,MD,1505,Nations League 2018-2020,Moldova vs San Marino
1504,Damașcan,[Vitalie Damaşcan],15.10.18,Moldova,MD,1506,Nations League 2018-2020,Belarus vs Moldova
1505,Drahun,[Stanislav Dragun],15.11.18,Belarus,BY,1509,Nations League 2018-2020,Luxembourg vs Belarus
1506,Drahun,[Stanislav Dragun],18.11.18,Belarus,BY,1510,Nations League 2018-2020,San Marino vs Belarus


In [22]:
combined_df_failed_3 = combined_df_failed_2[combined_df_failed_2['Status III'] != 'Working']
#combined_df_failed_2.loc[weird_indexes]
combined_df_failed_3

,Unnamed: 0,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case,Status II,Status III
1148,48,2681,"World Cup Qualifiers, Asia, 2022",Thailand vs Indonesia,03-06-2021,Thailand,TH,2021,FAIL,0,0.0,True,Lookup Required,False,0,E Amantegui,none,,
1164,64,2681,"World Cup Qualifiers, Asia, 2022",Thailand vs Indonesia,03-06-2021,Indonesia,ID,2021,FAIL,0,0,True,Lookup Required,False,0,W Sulaeman,none,,
1200,100,2683,"World Cup Qualifiers, Asia, 2022",Saudi Arabia vs Yemen,05-06-2021,Yemen,YE,2021,FAIL,0,0,True,Lookup Required,False,0,Ala Addin Mahdi,none,,
1259,159,2686,"World Cup Qualifiers, Asia, 2022",Turkmenistan vs North Korea,14-11-2019,Turkmenistan,TM,2019,FAIL,0,0,True,Lookup Required,False,0,W Ylýasow,none,,
1271,171,2686,"World Cup Qualifiers, Asia, 2022",Turkmenistan vs North Korea,14-11-2019,North Korea,KP,2019,FAIL,0,0,True,Lookup Required,False,0,Y Lee,none,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96402,636,1495,Nations League 2018-2020,Latvia vs Georgia,16.10.18,Georgia,GE,2018,FAIL,0,0.0,True,Lookup Required,False,0,Gviliya,none,,
96445,679,1497,Nations League 2018-2020,Andorra vs Georgia,15.11.18,Georgia,GE,2018,FAIL,0,0.0,True,Lookup Required,False,0,Gviliya,none,,
96479,713,1499,Nations League 2018-2020,Georgia vs Kazakhstan,19.11.18,Georgia,GE,2018,FAIL,0,0.0,True,Lookup Required,False,0,Gviliya,none,,
96585,819,1504,Nations League 2018-2020,Belarus vs Luxembourg,12.10.18,Belarus,BY,2018,FAIL,0,0.0,True,Lookup Required,False,0,Putsila,none,,


In [24]:
combined_df_failed_3.to_csv('lookup_required_dudes.csv')

In [23]:
combined_df_failed_3['ORIGINAL JERSEY'].value_counts().head(50)

#guys i know
#Natkho
#D Dakonam
#C Mafoumbi - Republic of Congo
#D NDinga - Republic of Congo 
#M Baudry - Republic of Congo
#T Alcantara - that's thiago 
#K Fortuné - Martinique
#Beto - probably the portugal guy

#rules 
#try to do a thing specific to republic of congo 

#maybe like a name match across the whole dataset? all nationalities? and super tight?

ORIGINAL JERSEY
Natkho                   21
D Dakonam                16
A-nan P                  13
D Hérelle                13
R Gomez                  12
O Færø                   12
C Mafoumbi               12
N Tolo                   12
Pico                     12
Zheng Zhi                11
R Yusef                  11
Jiang Guangtai           11
Gviliya                  11
Bapasy                   11
Ai Kesen                 11
Hahgverdi                11
Y Kavalyow               10
M Baudry                 10
D NDinga                  9
Benítez                   9
Faisal Saleh              9
A Al Faz                  9
CZeng                     9
C Andriamahitsinoro       9
K Parsemain               9
S Abaul                   9
T Alcantara               9
Ricketts                  8
S Crétinoir               8
Shelton                   8
XYang                     8
N El Shighail             8
J Delem                   7
Mc Kee Jaimes Anthony     7
M Islam Nazir             7
M Mo

# Online Lookup Required

In [25]:
combined_df_failed[combined_df_failed['Lookup Return Case'] == 'Online Lookup Required']


,Unnamed: 0,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case
36,36,3134,"World Cup Qualifiers, Europe, 2018",Norway vs Azerbaijan,01-09-2017,Azerbaijan,AZ,2017,FAIL,"['Rashad Sadyqov', 'Rashad Sadiqov']",0,True,Online Lookup Required,False,"['Rashad Sadyqov', 'Rashad Sadiqov']",R Sadqov,multiple
223,223,3143,"World Cup Qualifiers, Europe, 2018",Azerbaijan vs Northern Ireland,10-06-2017,Azerbaijan,AZ,2017,FAIL,"['Rashad Sadyqov', 'Rashad Sadiqov']",0,True,Online Lookup Required,False,"['Rashad Sadyqov', 'Rashad Sadiqov']",R Sadqov,multiple
1176,76,2682,"World Cup Qualifiers, Asia, 2022",Japan vs Saudi Arabia,01-02-2022,Japan,JP,2022,FAIL,Yuya Osako,0,True,Online Lookup Required,False,Yuya Osako,Y sako,single
1290,190,2687,"World Cup Qualifiers, Asia, 2022",Japan vs Vietnam,29-03-2022,Vietnam,VN,2022,FAIL,Thanh Hien Nguyen,0,True,Online Lookup Required,False,Thanh Hien Nguyen,Qu Ngc Hi,single
1296,196,2687,"World Cup Qualifiers, Asia, 2022",Japan vs Vietnam,29-03-2022,Vietnam,VN,2022,FAIL,Minh Tuan Ha,0,True,Online Lookup Required,False,Minh Tuan Ha,Phm Tun Hi,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95459,793,3270,"World Cup Qualifiers, Europe, 2022",Iceland vs Germany,08-09-2021,Iceland,IS,2021,FAIL,Bödvar Bödvarsson,0,True,Online Lookup Required,False,Bödvar Bödvarsson,B Sævarsson,single
95467,801,3270,"World Cup Qualifiers, Europe, 2022",Iceland vs Germany,08-09-2021,Iceland,IS,2021,FAIL,Helgi Danielsson,0,True,Online Lookup Required,False,Helgi Danielsson,Þ Helgason,single
95701,1035,3281,"World Cup Qualifiers, Europe, 2022",Iceland vs North Macedonia,05-09-2021,Iceland,IS,2021,FAIL,Bödvar Bödvarsson,0,True,Online Lookup Required,False,Bödvar Bödvarsson,B Sævarsson,single
95967,201,1476,Nations League 2018-2020,Lithuania vs Serbia,07.09.18,Lithuania,LT,2018,FAIL,Arunas Klimavicius,0.0,True,Online Lookup Required,False,Arunas Klimavicius,Ar Klimavičius,single


In [26]:
#guys with one name found - 1258
combined_df_failed_one_name_found = combined_df_failed[(combined_df_failed['Lookup Return Case'] == 'Online Lookup Required') & 
                   ((~combined_df_failed['Name'].apply(lambda x: isinstance(x, list))) &
                    (~combined_df_failed['Name'].str.startswith('['))) &
                   (combined_df_failed['Name(s) Found'] != '0')]#.tail(30)
combined_df_failed_one_name_found
#if the name is initial + name make sure the name starts with initial and ends with name. be mindful of the accents here if names are same w accents removed that's ok

#if its just name (no initials) do nothing 

#is the name that it found the closest match in the dataset_nationality?
#is there another name in the dataset thats basically a better quality of match?? or the same? 
    #if there is another potential name do you do the multi name match date lookup? 

        #if one guy is remaining YOU HAVE A MATCH

        #if multiple remaining, then see the below example when there's an array of names

        #if 0 remaining then what?

    #if not any other potential names, evaluate the quality of this match. 

        #if the quality is good then YOU HAVE A MATCH

        #if the quality is super super low then what?  

    #provided you have a name 
    #look up that name + that season 
    #if there are no rows, you have to look them up online 

,Unnamed: 0,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case
1176,76,2682,"World Cup Qualifiers, Asia, 2022",Japan vs Saudi Arabia,01-02-2022,Japan,JP,2022,FAIL,Yuya Osako,0,True,Online Lookup Required,False,Yuya Osako,Y sako,single
1290,190,2687,"World Cup Qualifiers, Asia, 2022",Japan vs Vietnam,29-03-2022,Vietnam,VN,2022,FAIL,Thanh Hien Nguyen,0,True,Online Lookup Required,False,Thanh Hien Nguyen,Qu Ngc Hi,single
1296,196,2687,"World Cup Qualifiers, Asia, 2022",Japan vs Vietnam,29-03-2022,Vietnam,VN,2022,FAIL,Minh Tuan Ha,0,True,Online Lookup Required,False,Minh Tuan Ha,Phm Tun Hi,single
1329,229,2689,"World Cup Qualifiers, Asia, 2022",Oman vs India,19-11-2019,Oman,OM,2019,FAIL,Mohsin Al Harbi,0,True,Online Lookup Required,False,Mohsin Al Harbi,Mohsin Al Khaldi,single
1366,266,2691,"World Cup Qualifiers, Asia, 2022",Cambodia vs Hong Kong,05-09-2019,Cambodia,KH,2019,FAIL,Ouk Sothy,0,True,Online Lookup Required,False,Ouk Sothy,Ouk Sovann,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95459,793,3270,"World Cup Qualifiers, Europe, 2022",Iceland vs Germany,08-09-2021,Iceland,IS,2021,FAIL,Bödvar Bödvarsson,0,True,Online Lookup Required,False,Bödvar Bödvarsson,B Sævarsson,single
95467,801,3270,"World Cup Qualifiers, Europe, 2022",Iceland vs Germany,08-09-2021,Iceland,IS,2021,FAIL,Helgi Danielsson,0,True,Online Lookup Required,False,Helgi Danielsson,Þ Helgason,single
95701,1035,3281,"World Cup Qualifiers, Europe, 2022",Iceland vs North Macedonia,05-09-2021,Iceland,IS,2021,FAIL,Bödvar Bödvarsson,0,True,Online Lookup Required,False,Bödvar Bödvarsson,B Sævarsson,single
95967,201,1476,Nations League 2018-2020,Lithuania vs Serbia,07.09.18,Lithuania,LT,2018,FAIL,Arunas Klimavicius,0.0,True,Online Lookup Required,False,Arunas Klimavicius,Ar Klimavičius,single


In [27]:
#guys with multiple names found - 321
combined_df_failed_multi_name_found = combined_df_failed[((combined_df_failed['Lookup Return Case'] == 'Online Lookup Required') & 
                    ((combined_df_failed['Name'].apply(lambda x: isinstance(x, list))) |
                     (combined_df_failed['Name'].str.startswith('['))))]#.tail(50)
combined_df_failed_multi_name_found

#if multiple names were found

    #if the Jersey has an initial 
        #keep all matches that start with the initial 

        #except if the initial is a lowercase d. then, make sure there is a word starting with d followed by the Name 
        #so if its d Silva make sure theres a word startign with d then Silva soemwhere in the name. 

    #sadiqov case - what if the names are the same???
        #in this case you should just look up each name and use like whichever has the record from the saeason of the game 

    #do the match date lookup thing with each of the matches to check who played in that game. 
        #if you still have more than 1 name

            #look at the success dataset to find the record of each remaining guy
                #if its ivanovic and you find both B and D Ivanovic played according to transfermarkt
                
                #check if there's a record of those guys in that match / date (in our dataset)
                    
                    #if all but one guy is in the success dataset, the guy who is not there is the name you need to use.

                    #if none remain after checking the dataset just remove this row. but print that out. that would be weird

                    #if multiple remain after checking the success dataset(there were 3 names remaining, only found 1 record)
                        #print this out this is a special case.

        #if you are just left with one name using the match date lookup 

            #that's a success - find that guy in the database

,Unnamed: 0,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case
36,36,3134,"World Cup Qualifiers, Europe, 2018",Norway vs Azerbaijan,01-09-2017,Azerbaijan,AZ,2017,FAIL,"['Rashad Sadyqov', 'Rashad Sadiqov']",0,True,Online Lookup Required,False,"['Rashad Sadyqov', 'Rashad Sadiqov']",R Sadqov,multiple
223,223,3143,"World Cup Qualifiers, Europe, 2018",Azerbaijan vs Northern Ireland,10-06-2017,Azerbaijan,AZ,2017,FAIL,"['Rashad Sadyqov', 'Rashad Sadiqov']",0,True,Online Lookup Required,False,"['Rashad Sadyqov', 'Rashad Sadiqov']",R Sadqov,multiple
2218,18,4198,"World Cup 2022, Qualifiers, CONCACAF",Jamaica vs Honduras,30-Mar-22,Honduras,HN,2022,FAIL,"['Carlos Will Mejia', 'Alfredo Mejia', 'Juan R...",0.0,True,Online Lookup Required,False,"['Carlos Will Mejia', 'Alfredo Mejia', 'Juan R...",Mejia,multiple
2227,27,4200,"World Cup 2014, Qualifiers, CONMEBOL",Ecuador vs Venezuela,07-Oct-11,Ecuador,EC,2011,FAIL,"['Antonio Valencia', 'Enner Valencia', 'Anthon...",0.0,True,Online Lookup Required,False,"['Antonio Valencia', 'Enner Valencia', 'Anthon...",Valencia,multiple
2235,35,4200,"World Cup 2014, Qualifiers, CONMEBOL",Ecuador vs Venezuela,07-Oct-11,Venezuela,VE,2011,FAIL,"['Franklin Lucena', 'Ronaldo Lucena']",0.0,True,Online Lookup Required,False,"['Franklin Lucena', 'Ronaldo Lucena']",Lucena,multiple
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89803,637,1291,Copa America 2019,Colombia vs Paraguay,23.06.19,Paraguay,PY,2019,FAIL,"['Luis Cardozo', 'Óscar Cardozo']",0,True,Online Lookup Required,False,"['Luis Cardozo', 'Óscar Cardozo']",O Cardozo,multiple
90037,871,1303,Copa America 2021,Peru vs Paraguay,03.07.21,Paraguay,PY,2021,FAIL,"['Juan Espínola', 'Alberto Espínola']",0,True,Online Lookup Required,False,"['Juan Espínola', 'Alberto Espínola']",Espínola,multiple
90385,119,2128,"World Cup Qualifiers, Africa, 2022",Mali vs Tunisia,25-03-2022,Mali,ML,2022,FAIL,"['Abdoulay Diaby', 'Alassane Diaby']",0,True,Online Lookup Required,False,"['Abdoulay Diaby', 'Alassane Diaby']",A Diaby,multiple
91222,956,2166,"World Cup Qualifiers, Africa, 2022",Guinea-Bissau vs Sudan,15-11-2021,Guinea-Bissau,GW,2021,FAIL,"['Arnaud Mendy', 'Alexandre Mendy']",0,True,Online Lookup Required,False,"['Arnaud Mendy', 'Alexandre Mendy']",A Mendy,multiple


In [28]:
#guys with 0 found - 235
combined_df_failed_0_names_found = combined_df_failed[(combined_df_failed['Lookup Return Case'] == 'Online Lookup Required') & 
                   ((~combined_df_failed['Name'].apply(lambda x: isinstance(x, list))) &
                    (~combined_df_failed['Name'].str.startswith('['))) &
                   (combined_df_failed['Name(s) Found'] == '0')]

combined_df_failed_0_names_found

,Unnamed: 0,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case
4574,174,2231,"World Cup Qualifiers, Africa, 2026",Mozambique vs Algeria,19-11-2023,Algeria,DZ,2023,FAIL,0,0,True,Online Lookup Required,False,0,A Gouiri,none
4636,236,2234,"World Cup Qualifiers, Africa, 2026",Zimbabwe vs Nigeria,19-11-2023,Nigeria,NG,2023,FAIL,0,0,True,Online Lookup Required,False,0,N Tella,none
12342,176,3647,"Gold Cup, 2013",Martinique vs Mexico,14-07-2013,Martinique,MQ,2013,FAIL,0,0,True,Online Lookup Required,False,0,K Olimpa,none
12344,178,3647,"Gold Cup, 2013",Martinique vs Mexico,14-07-2013,Martinique,MQ,2013,FAIL,0,0,True,Online Lookup Required,False,0,G Arnolin,none
12345,179,3647,"Gold Cup, 2013",Martinique vs Mexico,14-07-2013,Martinique,MQ,2013,FAIL,0,0,True,Online Lookup Required,False,0,J Babin,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93079,613,3717,"Gold Cup, 2017",United States vs Martinique,13-07-2017,Martinique,MQ,2017,FAIL,0,0,True,Online Lookup Required,False,0,J Audel,none
93294,828,3728,"Gold Cup, 2019",Cuba vs Martinique,20-06-2019,Martinique,MQ,2019,FAIL,0,0,True,Online Lookup Required,False,0,J Babin,none
93297,831,3728,"Gold Cup, 2019",Cuba vs Martinique,20-06-2019,Martinique,MQ,2019,FAIL,0,0,True,Online Lookup Required,False,0,J Marveaux,none
93415,949,3734,"Gold Cup, 2019",Martinique vs Mexico,24-06-2019,Martinique,MQ,2019,FAIL,0,0,True,Online Lookup Required,False,0,J Babin,none


In [29]:
combined_df_failed_one_name_found.to_csv('Online lookup required - 1 name found.csv')
combined_df_failed_multi_name_found.to_csv('Online lookup required - multiple names found.csv')
combined_df_failed_0_names_found.to_csv('Online lookup required - 0 names found.csv')

# other cases below

### cyrillic


In [33]:
combined_df_cyrillic = combined_df[combined_df['Lookup Return Case'] == 'not done yet']
combined_df_cyrillic.to_csv('cyrillic_dudes.csv')

### others still

In [10]:
combined_df_failed[combined_df_failed['Lookup Return Case'].str.startswith("FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS: ")]

,Unnamed: 0,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case
1018,1018,3179,"World Cup Qualifiers, Europe, 2018",Liechtenstein vs Italy,12-11-2016,Liechtenstein,LI,2016,FAIL,"['Martin Büchel', 'Marcel Büchel']",0,True,FILTERING USING MATCH DATE LEFT MULTIPLE RESUL...,False,"['Martin Büchel', 'Marcel Büchel']",M Büchel,multiple
1020,1020,3179,"World Cup Qualifiers, Europe, 2018",Liechtenstein vs Italy,12-11-2016,Liechtenstein,LI,2016,FAIL,"['Martin Büchel', 'Marcel Büchel']",0,True,FILTERING USING MATCH DATE LEFT MULTIPLE RESUL...,False,"['Martin Büchel', 'Marcel Büchel']",M Büchel,multiple
1447,347,2694,"World Cup Qualifiers, Asia, 2022",Kyrgyzstan vs Tajikistan,19-11-2019,Tajikistan,TJ,2019,FAIL,"['Davronzhon Ergashev', 'Dzhakhongir Ergashev'...",0,True,FILTERING USING MATCH DATE LEFT MULTIPLE RESUL...,False,"['Davronzhon Ergashev', 'Dzhakhongir Ergashev'...",D Ergashev,multiple
1484,384,2696,"World Cup Qualifiers, Asia, 2022",Malaysia vs Thailand,14-11-2019,Malaysia,MY,2019,FAIL,"['Syamim Yahya', 'Syahrul Azwari', 'Syawal Nor...",0,True,FILTERING USING MATCH DATE LEFT MULTIPLE RESUL...,False,"['Syamim Yahya', 'Syahrul Azwari', 'Syawal Nor...",Sya,multiple
1689,589,2706,"World Cup Qualifiers, Asia, 2026",Laos vs Nepal,17-10-2023,Nepal,NP,2023,FAIL,"['Anjan Bista', 'Arik Bista']",0,True,FILTERING USING MATCH DATE LEFT MULTIPLE RESUL...,False,"['Anjan Bista', 'Arik Bista']",A Bista,multiple
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96142,376,1484,Nations League 2018-2020,Serbia vs Montenegro,17.11.18,Serbia,RS,2018,FAIL,"['Nikola Milenkovic', 'Milos Veljkovic', 'Mila...",0.0,True,FILTERING USING MATCH DATE LEFT MULTIPLE RESUL...,False,"['Nikola Milenkovic', 'Milos Veljkovic', 'Mila...",Milenković,multiple
96187,421,1486,Nations League 2018-2020,Serbia vs Lithuania,20.11.18,Serbia,RS,2018,FAIL,"['Nikola Milenkovic', 'Milos Veljkovic', 'Mila...",0.0,True,FILTERING USING MATCH DATE LEFT MULTIPLE RESUL...,False,"['Nikola Milenkovic', 'Milos Veljkovic', 'Mila...",Milenković,multiple
96295,529,1491,Nations League 2018-2020,Andorra vs Kazakhstan,10.09.18,Andorra,AD,2018,FAIL,"['Jordi Rubio', 'Jesús Rubio']",0.0,True,FILTERING USING MATCH DATE LEFT MULTIPLE RESUL...,False,"['Jordi Rubio', 'Jesús Rubio']",Ch Rubio,multiple
96328,562,1492,Nations League 2018-2020,Georgia vs Andorra,13.10.18,Andorra,AD,2018,FAIL,"['Jordi Rubio', 'Jesús Rubio']",0.0,True,FILTERING USING MATCH DATE LEFT MULTIPLE RESUL...,False,"['Jordi Rubio', 'Jesús Rubio']",Ch Rubio,multiple


In [4]:
combined_df['Status'].value_counts()

Status
SUCCESS    78063
FAIL       18803
Name: count, dtype: int64